## Ngày 5: Content-based filtering (Lọc dựa trên Nội dung)

Một trong những phương pháp “cơ bản” nhưng vô cùng mạnh mẽ trong hệ thống đề xuất – **Content-Based Filtering**. Đây là phương pháp gợi ý dựa trên nội dung, nghĩa là nếu bạn từng thích một món ăn nào đó, hệ thống sẽ gợi ý món ăn có thành phần, hương vị tương tự.


### Lý thuyết
- Khái niệm: Content-based filtering (CBF) tập trung vào các đặc trưng (features) của sản phẩm hay nội dung. Ví dụ, với phim, ta có thể dùng thể loại, đạo diễn, diễn viên, từ khóa mô tả, v.v. Nếu người dùng đã thích một bộ phim có đặc trưng "Hành động" và "Phiêu lưu", thì hệ thống sẽ tìm những bộ phim có đặc trưng tương tự để gợi ý.

- Quy trình cơ bản:
    + Trích xuất đặc trưng: Sử dụng các kĩ thuật như TF-IDF, CountVectorizer, Word2Vec,... để chuyển đổi mô tả sản phẩm thành vector số
    + Đo lường đọ tương đồng: Thông thường, dùng cosine similarity để tính khoảng cách giữa các vector đặc trưng
    + Gợi ý sản phẩm: dựa vào các độ tương đồng, đề xuất những sản phẩm có nội dung gần giống với sản phẩm mà người dùng đã thích

- Ưu điểm:
    + Giải quyết bài toán "cold start" cho các sản phẩm mới (nếu có thông tin mô tả)
    + Không phụ thuộc vào phản hồi của người dùng khác, chỉ cần dựa vào nội dung sản phẩm

- Hạn chế:
    + Hạn chế trong việc khám phá các "sự đa dạng" nếu người dùng có sở thích phong phú
    + Cần có mô tả hoặc đặc trung rõ ràng cho từng sản phẩm

In [1]:
# Content-based filtering dựa trên thể loại của phim
# Tạo một CountVectorizer để chuyển đổi thể loại thành vector
# Tính toán độ tương đồng giữa các phim bằng cosine similarity

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc dữ liệu
movies = pd.read_csv("../Datasets/MovieLens/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# Lấy 100 phim đầu tiên tránh tràn bộ nhớ
small_movies = movies[:100]

# Sử dụng CountVectorizer để chuyển dổi cột genres thành vector
# Cũng lưu ý rằng chúng ta dùng tokenizer riêng để tác các thể loại bằng dấu "|"
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(small_movies['genres'])

# Tính toán cosine similarity giữa các phim
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)
print(f'Ma trận cosine similarity: \n{cosine_sim}')

# Giải thích:
# CountVectorizer: Biến thể loại phim thành ma trận số, mỗi cột ứng với một thể loại
# Cosine Similarity: Tính toán độ tương đồng giữa các vector của phim, giá trị càng cao cho thấy phim càng giống nhau về nội dung

# Khi người dùng thích một phim nào đó, hệ thống có thể dựa vào ma trận cosine similarity để tìm ra các
# phim có độ tương đồng cao với phim đó. Ví dụ nếu người dùng thích phim A thì hệ thống sẽ gợi ý các phim
# có giá trị cosine similarity với phim AA

# Hiểu rõ các đặc trưng của sản phẩm: Dữ liệu đầu vào càng chất, kết quả gợi ý càng chính xác. Hãy chú ý đến việc lựa chọn các đặc trưng phù hợp
# Thử nghiệm với các vectorizer khác: TF-IDF có thể cho kết quả khác so với CountVectorizer, tùy vào bài toán cụ thể
# Kết hợp kiến thức: Content-based filtering thường được kết hợp với Collaborative filtering trong hệ thống Hybrid để 
# khai thác tối đa thông tin từ cả nội dung và hành vi người dùng

Ma trận cosine similarity: 
[[1.         0.77459667 0.31622777 ... 0.         0.         0.4472136 ]
 [0.77459667 1.         0.         ... 0.         0.         0.28867513]
 [0.31622777 0.         1.         ... 0.         0.         0.70710678]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.4472136  0.28867513 0.70710678 ... 0.         0.         1.        ]]


e:\LEARNING\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Giải thích cosine_sim
# Hãy nhìn vào các giá trị sau đây:
print(cosine_sim.shape)
print(cosine_sim[13].shape) # 13 là ngẫu nhiên
# => Từ kết quả in ra ta thấy rằng mỗi một bộ phim được tính toán độ tương đương với tất cả các phim còn lại

(100, 100)
(100,)


In [8]:
genres = movies['genres']
genres_types = set()
for genre in genres:
    for g in genre.split('|'):
        genres_types.add(g)

print(len(genres_types)) # Total 20 genres
print(genre_matrix.shape) # 100 films with 17 genres

20
(100, 17)


### Công thức Cosine Similarity
cos(*Theta*) = (A.B)/(||A||.||B||)

In [10]:
# Tính cosine similarity bằng numpy
import numpy as np
from numpy.linalg import norm

A = np.array([1, 8])
B = np.array([9, 2])

cos_sim = np.dot(A, B)/(norm(A)*norm(B))
print(cos_sim)

0.33633639699815626


In [13]:
# Tính cosine similarity bằng sklearn
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(A.reshape(1, -1), B.reshape(1, -1))
print(cos_sim)
print(cosine_similarity(A, B)) # => Error:
# ValueError: Expected 2D array, got 1D array instead:
# array=[1. 8.].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

[[0.3363364]]


In [15]:
# Tìm top 3, 5, 10 phim có sự tương đồng lớn nhất so với một phim tùy chọn bất kỳ
def get_similar_movies(movie_id, top_n):
    # Lấy index của phim trong bảng
    movie_idx = small_movies[small_movies['movieId'] == movie_id].index[0]

    # Lấy các giá trị cosine similarity giữa phim này và các phim khác
    sim_scores = list(enumerate(cosine_sim[movie_idx]))

    # Sắp xếp các phim theo độ tương đồng
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Lấy top N phim
    top_sim_scores = sim_scores[1:top_n+1]

    # Lấy thông tin của các phim
    movie_indices = [i[0] for i in top_sim_scores]
    return small_movies.iloc[movie_indices]

# In ra top 3 phim tương đồng với phim có id = 1
print(get_similar_movies(1, 3))

    movieId                           title                             genres
55       56  Kids of the Round Table (1995)  Adventure|Children|Comedy|Fantasy
1         2                  Jumanji (1995)         Adventure|Children|Fantasy
12       13                    Balto (1995)       Adventure|Animation|Children


In [16]:
# In ra top 5 phim tương đồng với phim có id = 2
print(get_similar_movies(2, 5))

    movieId  ...                                       genres
59       60  ...                   Adventure|Children|Fantasy
55       56  ...            Adventure|Children|Comedy|Fantasy
7         8  ...                           Adventure|Children
0         1  ...  Adventure|Animation|Children|Comedy|Fantasy
12       13  ...                 Adventure|Animation|Children

[5 rows x 3 columns]


In [ ]:
# In ra top 10 phim tương đồng với phim có id = 3
print(get_similar_movies(3, 10))

    movieId                               title                 genres
6         7                      Sabrina (1995)         Comedy|Romance
38       39                     Clueless (1995)         Comedy|Romance
63       64                Two if by Sea (1996)         Comedy|Romance
67       68  French Twist (Gazon maudit) (1995)         Comedy|Romance
3         4            Waiting to Exhale (1995)   Comedy|Drama|Romance
10       11      American President, The (1995)   Comedy|Drama|Romance
51       52             Mighty Aphrodite (1995)   Comedy|Drama|Romance
57       58   Postman, The (Postino, Il) (1994)   Comedy|Drama|Romance
91       93          Vampire in Brooklyn (1995)  Comedy|Horror|Romance
92       94              Beautiful Girls (1996)   Comedy|Drama|Romance


In [24]:
# Một người có sở thích xem phim các thể loại như: Romance, Adventure, Science, Fiction, Thriller
# Hãy tìm ra top 10 phim mà người đó có thể sẽ thích
def recommend_movies(genres, top_n):
    # Tạo một vector từ thể loại của người dùng
    user_vector = vectorizer.transform([genres])

    # Tính toán độ tương đồng giữa vector của người dùng và các phim
    user_sim = cosine_similarity(user_vector, genre_matrix)

    # Lấy top N phim
    top_sim_scores = sorted(enumerate(user_sim[0]), key=lambda x: x[1], reverse=True)[:top_n]

    # Lấy thông tin của các phim
    movie_indices = [i[0] for i in top_sim_scores]
    return small_movies.iloc[movie_indices]

# In ra top 10 phim mà người dùng có thể sẽ thích
print(recommend_movies('Romance|Adventure|Science Fiction|Thriller', 10))

    movieId                         title                          genres
9        10              GoldenEye (1995)       Action|Adventure|Thriller
14       15       Cutthroat Island (1995)        Action|Adventure|Romance
32       33       Wings of Courage (1995)          Adventure|Romance|IMAX
93       95           Broken Arrow (1996)       Action|Adventure|Thriller
99      101          Bottle Rocket (1996)  Adventure|Comedy|Crime|Romance
2         3       Grumpier Old Men (1995)                  Comedy|Romance
6         7                Sabrina (1995)                  Comedy|Romance
7         8           Tom and Huck (1995)              Adventure|Children
16       17  Sense and Sensibility (1995)                   Drama|Romance
24       25      Leaving Las Vegas (1995)                   Drama|Romance
